In [1]:
from predict import predict_from_filepath
import tensorflow as tf
from loss import dice_coef, dice_loss
from model import unet_3d
from display_widget import display_prediction
import nibabel as nib
import numpy as np
from os.path import dirname, basename


In [11]:
# File paths
# model_path = "cerebellum_model.h5"
image_path = r'data\cerebellum_data\image\tc1_272718-ob_c.nii.gz'
model_name = "cerebellum_model"
model_path = f"models/{model_name}.h5"
gt_path = image_path.replace("image", "label")

# Load in the image and ground truth
image = nib.load(image_path).get_fdata()
gt = nib.load(gt_path).get_fdata()
gt[gt != 0] = 1



In [12]:
# Predict on whole image
pred = predict_from_filepath(model_path, image_path).astype(np.float64)



3/3 [==============================] - 1s 123ms/step


In [13]:
# Display prediction
display_prediction(image, gt, pred)




interactive(children=(Dropdown(description='axes', options=('xy', 'yz', 'xz'), value='xy'), IntSlider(value=0,…

array([0.])